In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
from scipy import sparse
from scipy.spatial import distance
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def Construct(Coo, Typ, Ener):
    #Read coordinates from file
    coords = list(map(tuple, np.genfromtxt(Coo, delimiter=" ")))
    #Build Graph
    F=nx.Graph()
    F.add_nodes_from(coords)
    #Read Attributes
    types = np.genfromtxt(Typ, delimiter=" ")
    energies = np.genfromtxt(Ener, delimiter=" ")
    #Set Attributes
    i=0
    for u in F.nodes():
        F.nodes[u]["pos"]=u
        F.nodes[u]["type"]=types[i]
        F.nodes[u]["potential"]=energies[i]
        i+=1  
    #Construct Edges
    F.add_edges_from(nx.geometric_edges(F, 1))
    
    ## Inizializing Marcus Rates ##
   
    for (u, v) in F.edges(): #die beiden Endpunkte u und v der Kante 
        v0=1 ## attempt frequency (largest possible rate) (in KMC normalized for all Hopping probabilities on a site)
        alpha=1 ## charge delocalization constant
        dist=distance.euclidean(F.nodes[u]["pos"] , F.nodes[v]["pos"]) ## distantce between hopping sites
        Er=1  ## Reorganization Energy
        kbT=1
        deltaE=F.nodes[u]["potential"] - F.nodes[v]["potential"] ## v is start and u is end
        if deltaE <0: ## flow from high potential to low
            F.edges[u,v]['weight'] = v0 * np.exp(-2* alpha * dist) * np.exp(-((deltaE - Er)**2)/(4* Er * kbT))
        else: ## reorient edge
            # Could not just delete edge and add oposite as (Non-Di-)Graphs sort Edges by the sequence of nodes
            # Therefore I just add a negative sign to the resistor and thereby flipping the current direction
            F.edges[u,v]['weight'] = (-1) *v0 * np.exp(-2* alpha * dist) * np.exp(-((deltaE - Er)**2)/(4* Er * kbT))
        
        
    ## These Edges should maybe be oriented by using some variation of the code in the end ##
    
    return F


#Gitter=Construct("coord_0.dat", "mol_types_0.dat", "site_energies_0.dat")

In [3]:
class Solve:
    def __init__(self, X):
        self.G=X
        self.N=self.G.order()
        self.nR=self.G.number_of_edges()
        
    def resistor(self):
        #Extract the values of the resistors from the graph and build a nR x nR matrix
        mat=sparse.spdiags(list(nx.get_edge_attributes(self.G, "weight").values()), 0, self.nR, self.nR)
        return sparse.csc_matrix(mat)
    
    def incidence(self):
        #Builds the incidence matrix from the graph
        mat= np.transpose(nx.incidence_matrix(self.G, oriented=1)) #Beachte Transpose damit die Dimensionen der networkx funktion zum paper passen
        
        
        ## Siehe Codeschnipsel im ersten Beispiel falls die Orientierung ungünstig ist ##
        
        
        return mat
    
    def voltages(self):
        #Get the potential values from the nodes and build a vector
        vec=np.array(list(nx.get_node_attributes(self.G, "potential").values()))
        return vec

    
    def currents(self):
        #Combines the other functions to get the currents trough the resistors
        return - (sparse.linalg.inv(self.resistor()) @ self.incidence()) @ self.voltages()

    
#Solve(Gitter).currents()   